In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [3]:
########################################## Larger Dataset (US) ###############################################################

US_df = pd.read_csv('US_Monthly_2011.csv')
# US_df = US_df.sort_values(by = ['rid'])
# print("Count values before the drop: ")
# print(US_df['rid'].value_counts())
# print(US_df.isnull().sum().sum())
US_df = US_df.dropna()
# print("Count values after the drop: ")
# print(US_df['rid'].value_counts())


Source_US_df = US_df.loc[US_df['rid'].isin(['3', '4', '5', '9'])]
Target_US_df = US_df.loc[US_df['rid'].isin(['1','2','6','7','8'])]

train_Target_US_df = Target_US_df.loc[Target_US_df['rid'].isin(['2','7','8'])]
test_Target_US_df = Target_US_df.loc[Target_US_df['rid'].isin(['1','6'])]

drop_rid = ['rid']
Source_US_df = Source_US_df.drop(drop_rid, axis =1)
train_Target_US_df = train_Target_US_df.drop(drop_rid, axis =1)
test_Target_US_df = test_Target_US_df.drop(drop_rid, axis =1)

target_column = ['pm25_value']
Source_US_df_y = Source_US_df[target_column]
Source_US_df_X = Source_US_df.drop(target_column, axis = 1)

train_Target_US_df_y = train_Target_US_df[target_column]
train_Target_US_df_X = train_Target_US_df.drop(target_column, axis = 1)

test_Target_US_df_y = test_Target_US_df[target_column]
test_Target_US_df_X = test_Target_US_df.drop(target_column, axis = 1)

TF_train_X = pd.concat([Source_US_df_X, train_Target_US_df_X], sort= False)
TF_train_y = pd.concat([Source_US_df_y, train_Target_US_df_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()
np_test_Target_US_df_X = test_Target_US_df_X.to_numpy()
np_test_Target_US_df_y = test_Target_US_df_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_test_Target_US_df_y_list = np_test_Target_US_df_y.ravel()

print(Source_US_df_X.shape)
print(train_Target_US_df_X.shape)

(5353, 62)
(2881, 62)


In [32]:
########################################## Shorter Individual Datasets (US) ###############################################################


ss = StandardScaler()

US_df_train = pd.read_csv('US_data/US_10_slice/Train/Train_5.csv')
US_train_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid', 'clust']
US_df_train = US_df_train.drop(US_train_droplist, axis = 1)
US_df_train = US_df_train.rename(columns={"is": "ISS"})

US_cols = US_df_train.columns.difference(['pm25_value'])

US_df_train[US_cols] = ss.fit_transform(US_df_train[US_cols])
print(US_df_train.shape)

US_df_transfer = pd.read_csv('US_data/US_10_slice/Transfer/Transfer_5.csv')
US_transfer_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid', 'clust']
US_df_transfer = US_df_transfer.drop(US_transfer_droplist, axis = 1)
US_df_transfer = US_df_transfer.rename(columns={"is": "ISS"})

US_df_transfer[US_cols] = ss.fit_transform(US_df_transfer[US_cols])
print(US_df_transfer.shape)

US_df_test = pd.read_csv('US_data/US_10_slice/Test/Test_5.csv')
US_test_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid']
US_df_test = US_df_test.drop(US_test_droplist, axis = 1)
US_df_test = US_df_test.rename(columns={"is": "ISS"})

US_df_test[US_cols] = ss.fit_transform(US_df_test[US_cols])
print(US_df_test.shape)


############## Split into predictor and features ################
target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

############ Merge and make a numpy array #########################
TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()


(153, 19)
(3442, 19)
(1310, 19)


In [35]:
################################## Finding best instances from the source dataset (US) ######################################################

US_df_transfer["ManDis"] = ""

US_df_train_mean = []
prow = US_df_train.mean()
US_df_train_mean = [prow.pm25_value, prow.narr_dpt, prow.narr_vis, prow.nldas_pevapsfc, prow.nldas_dlwrfsfc, prow.nldas_dswrfsfc, 
                          prow.nldas_cape, prow.nldas_pressfc, prow.nldas_tmp2m, prow.nldas_rh2m, prow.nldas_ugrd10m, prow.nldas_vgrd10m,
                          prow.forest_cover, prow.elev, prow.emissi11, prow.local, prow.ISS, prow.pd, prow.gc_aod]

rowidx = 0

for row in US_df_transfer.itertuples():
    row_list =[row.pm25_value, row.narr_dpt, row.narr_vis, row.nldas_pevapsfc, row.nldas_dlwrfsfc, row.nldas_dswrfsfc, 
            row.nldas_cape, row.nldas_pressfc, row.nldas_tmp2m, row.nldas_rh2m, row.nldas_ugrd10m, row.nldas_vgrd10m,
            row.forest_cover, row.elev, row.emissi11, row.local, row.ISS, row.pd, row.gc_aod]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = US_df_train_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    US_df_transfer.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

US_df_transfer = US_df_transfer.sort_values(by =['ManDis'])
US_df_transfer = US_df_transfer.head(714) 
US_df_transfer = US_df_transfer.drop(['ManDis'], axis =1)
US_df_transfer = US_df_transfer.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", US_df_train.shape)
print("Source Set: ", US_df_transfer.shape)
print("Test Set: ", US_df_test.shape)


target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()

print("---------------------------")

###############################################################################################################################################

Target Set:  (153, 19)
Source Set:  (714, 19)
Test Set:  (1310, 19)
---------------------------


In [37]:
####################### STrAdaBoost.R2 #########################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

# sample_size = [len(US_df_transfer_X), len(US_df_train_X)]
sample_size = [714, 153]

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

print("STrAdaBoost.R2 AS")
print("-------------------------------------------")

r2scorelist_stradaboost_us = []
rmselist_stradaboost_us = []

for x in range(0, 10):

    model_stradaboost_us = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                          n_estimators = n_estimators, sample_size = sample_size,
                          steps = steps, fold = fold,
                          random_state = random_state)

    model_stradaboost_us.fit(np_TF_train_X, np_TF_train_y_list)
    y_pred_us = model_stradaboost_us.predict(np_US_df_test_X)

    mse_stradaboost_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_us))
    rmselist_stradaboost_us.append(mse_stradaboost_us)

    r2_score_stradaboost_us = pearsonr(np_US_df_test_y_list, y_pred_us)
    r2_score_stradaboost_us = (r2_score_stradaboost_us[0])**2
    r2scorelist_stradaboost_us.append(r2_score_stradaboost_us)
    
print("RMSE of StrAdaboost:", rmselist_stradaboost_us)
print("R^2 of StrAdaboost:", r2scorelist_stradaboost_us)

print("\n")
    
print("RMSE of StrAdaboost:", statistics.mean(rmselist_stradaboost_us), statistics.stdev(rmselist_stradaboost_us))
print("R^2 of StrAdaboost:", statistics.mean(r2scorelist_stradaboost_us), statistics.stdev(r2scorelist_stradaboost_us))

print("-------------------------------------------")

STrAdaBoost.R2 AS
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE of StrAdaboost: [5.729734963498598, 5.804636863702339, 5.6699154873885105, 6.025391937562782, 5.80057033331383, 5.6649735569851005, 5.976982997493459, 5.720269574604312, 5.880813960250914, 5.730236637922472]
R^2 of StrAdaboost: [0.168297767321497, 0.14940461580334483, 0.1856700294292351, 0.10118219708460188, 0.14500331848798168, 0.1885604574932414, 0.1086757328862196, 0.17342487626334976, 0.12711076570335983, 0.16523277525726074]


RMSE of StrAdaboost: 5.800352631272232 0.12471797023674308
R^2 of StrAdaboost: 0.15125625357300918 0.0307302963055734

In [33]:
####################### GBR Transfer Learning #########################################################
print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBRTL_us = []
rmselist_GBRTL_us = []

for x in range(0, 10):

    model_GBRTL_us = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_us.fit(np_TF_train_X, np_TF_train_y_list)


    y_pred_GBRTL_us = model_GBRTL_us.predict(np_US_df_test_X) 

    rmse_GBRTL_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_GBRTL_us))
    rmselist_GBRTL_us.append(rmse_GBRTL_us)
    
    r2_score_GBRTL_us = pearsonr(np_US_df_test_y_list, y_pred_GBRTL_us)
    r2_score_GBRTL_us = (r2_score_GBRTL_us[0])**2
    r2scorelist_GBRTL_us.append(r2_score_GBRTL_us)
    
print("RMSE of GBR:", rmselist_GBRTL_us)
print("R^2 of GBR:", r2scorelist_GBRTL_us)

print("\n")

print("RMSE of GBR:", statistics.mean(rmselist_GBRTL_us), statistics.stdev(rmselist_GBRTL_us))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBRTL_us), statistics.stdev(r2scorelist_GBRTL_us))

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------
RMSE of GBR: [5.340775527679069, 5.426111780305621, 5.471787607907473, 5.427058538518495, 5.387645786279188, 5.650975705744706, 5.466504809368625, 5.405203574316431, 5.328619349366162, 5.378858350707841]
R^2 of GBR: [0.2684925416685895, 0.24787417031492454, 0.24535611992198594, 0.2537526063094502, 0.2553581767122477, 0.21340487983645212, 0.24746856029039668, 0.2556984120419787, 0.27171623225328023, 0.26007186993351367]


RMSE of GBR: 5.428354103019361 0.09138988620992623
R^2 of GBR: 0.25191935692828193 0.01607320712942802
-------------------------------------------


In [34]:
####################### AdaBoost TL #########################################################
print("Adaptive Boosting Transfer Regression")
print("-------------------------------------------")

r2scorelist_Ada_us = []
rmselist_Ada_us = []

for x in range(0, 10):
    
    model_Ada_us = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_Ada_us.fit(np_TF_train_X, np_TF_train_y_list)

    y_pred_Ada_us = model_Ada_us.predict(np_US_df_test_X)
    
    rmse_Ada_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_Ada_us))
    rmselist_Ada_us.append(rmse_Ada_us)
    
    r2_score_Ada_us = pearsonr(np_US_df_test_y_list, y_pred_Ada_us)
    r2_score_Ada_us = (r2_score_Ada_us[0])**2
    r2scorelist_Ada_us.append(r2_score_Ada_us)

print("RMSE of Ada TL:", rmselist_Ada_us)
print("R^2 of Ada TL:", r2scorelist_Ada_us)

print("\n")
    
print("RMSE of Ada TL:", statistics.mean(rmselist_Ada_us), statistics.stdev(rmselist_Ada_us))
print("R^2 of Ada TL:", statistics.mean(r2scorelist_Ada_us), statistics.stdev(r2scorelist_Ada_us))

print("-------------------------------------------")

Adaptive Boosting Transfer Regression
-------------------------------------------
RMSE of Ada TL: [5.747322921182811, 5.8057066183958845, 5.66840828986415, 5.682139649401526, 5.683500710331262, 5.68877378822928, 5.7158050380774625, 5.732669488512051, 5.691906316240544, 5.717902628936855]
R^2 of Ada TL: [0.18258223085087244, 0.14697368182965317, 0.19340897563867754, 0.18076452288646003, 0.17899511906268573, 0.18429778023297783, 0.17822771976905247, 0.16589719436897546, 0.18462884865254206, 0.17592551773366158]


RMSE of Ada TL: 5.713413544917183 0.04086903932826141
R^2 of Ada TL: 0.17717015910255582 0.012724644443972732
-------------------------------------------
